# Integration of genetic determinants and GSM: gene essentiality on multiple carbon sources

### load module

In [1]:
import cobra
import re
import sys
import os
import json
from cameo import models
import pandas as pd
import numpy as np

### load GSM model from bigg database using CAMEO

In [2]:
m = models.bigg.iML1515

# gene dictionaries for mapping between ID and names
gene_name_dic = {}
for g in m.genes:
    gene_name_dic[g.name] = g.id
    
gene_name_dic2 = {}
for g in m.genes:
    gene_name_dic2[g.id] = g.name




### load genetic determinants

In [5]:
df_AB_all = pd.read_pickle('Data/GSMGeneticDeterminants_bothdir.pkl')
genes_ALL = list(df_AB_all.index)

# 1. Gene essentiality on rich media

In [ ]:
m.reactions.EX_o2_e.bounds = (-18.5, 0.0)

# get the carbon sources from the model
listSources=[]
for r in m.reactions:
    if 'EX_' in r.id:
        listSources.append(r.id)
                
m.reactions.EX_glc__D_e.lower_bound = 0.0
m.reactions.get_by_id('EX_o2_e').lower_bound = -18.5
medium = m.medium


# run in rich media
for source in listSources:
    m.reactions.get_by_id(source).lower_bound=-1.0



# get values for WT
    
try:
    sol_bio = m.optimize()
    if sol_bio.status != 'infeasible':
        WT_growth = sol_bio['BIOMASS_Ec_iML1515_core_75p37M']
    else:
        WT_growth = 0.0

except:
    WT_growth = 0.0
    

# run for all knockouts
KO_growth_rich = {}
for gene in genes_ALL:
    print(gene)
    with m: 
        m.genes.get_by_id(gene).knock_out() 
        
        try:
            sol_bio = m.optimize()
            if sol_bio.status != 'infeasible':
                KO_growth_rich[gene] = sol_bio['BIOMASS_Ec_iML1515_core_75p37M']
            else:
                KO_growth_rich[gene] = 0.0

        except:
            KO_growth_rich[gene.id] = 0.0
    
          
    







In [ ]:
growth_limiting_rich = []
count = 0
for i, j in KO_growth_rich.items():
    if j < 0.00001:
        print(gene_name_dic2[i])
        growth_limiting_rich.append(i)

print(len(growth_limiting_rich))



# 2. Gene essentiality on minimal media and auxotrophic behaviour

In [ ]:
m = models.bigg.iML1515

# gene dictionaries for mapping between ID and names
gene_name_dic = {}
for g in m.genes:
    gene_name_dic[g.name] = g.id
    
gene_name_dic2 = {}
for g in m.genes:
    gene_name_dic2[g.id] = g.name

# aerobic - RUN ON HPC USING ARRAYS TO SAVE TIME
m.reactions.EX_o2_e.bounds = (-18.5, 0.0)


                
m.reactions.EX_glc__D_e.lower_bound = -10.0
m.reactions.get_by_id('EX_o2_e').lower_bound = -18.5
medium = m.medium



# get values for WT
    
try:
    sol_bio = m.optimize()
    if sol_bio.status != 'infeasible':
        WT_growth = sol_bio['BIOMASS_Ec_iML1515_core_75p37M']
    else:
        WT_growth = 0.0

except:
    WT_growth = 0.0
    

# run for all knockouts
KO_growth = {}
for gene in genes_ALL:
    print(gene)
    with m: 
        m.genes.get_by_id(gene).knock_out() 
        
        try:
            sol_bio = m.optimize()
            if sol_bio.status != 'infeasible':
                KO_growth[gene] = sol_bio['BIOMASS_Ec_iML1515_core_75p37M']
            else:
                KO_growth[gene] = 0.0

        except:
            KO_growth[gene] = 0.0
    
          
    








In [ ]:
growth_limiting_minimal = []
count = 0
for i, j in KO_growth.items():
    if j < 0.00001:
        if i not in growth_limiting_rich:
            print(gene_name_dic2[i], j)
            growth_limiting_minimal.append(i)
            count += 1

print(len(growth_limiting_minimal))


In [2]:
### Auxotrophy

In [ ]:
# check for auxotrophy
m = models.bigg.iML1515

m.reactions.EX_o2_e.bounds = (-18.5, 0.0)

# get the carbon sources from the model
listSources=[]
for r in m.reactions:
    if 'EX_' in r.id:
        listSources.append(r.id)
                
#m.reactions.EX_glc__D_e.lower_bound = -1.0
m.reactions.get_by_id('EX_o2_e').lower_bound = -18.5
medium = m.medium




CgrowthCapabilities_auxo = pd.DataFrame(index=listSources)

# run for all knockouts
count = 0
for gene in growth_limiting:
    print(gene)
    
    with m: 
        col = []
        #gene_id = gene_name_dic[gene]
        m.genes.get_by_id(gene).knock_out() 
        
        for source in listSources:
            m.reactions.get_by_id(source).lower_bound=-10.0


            try:
                sol_bio = m.optimize()
                if sol_bio.status != 'infeasible':
                    col.append(sol_bio['BIOMASS_Ec_iML1515_core_75p37M'])
                else:
                    col.append(0.0)

            except:
                col.append(0.0)
    
    

            # turn transporter off
            if source not in medium.keys():
                m.reactions.get_by_id(source).lower_bound = 0.0
        
        CgrowthCapabilities_auxo.insert(count, column = gene, value = col)



    

# 3. Alternative carbon source utilisation

In [ ]:
# aerobic - RUN ON HPC USING ARRAYS TO SAVE TIME
m.reactions.EX_o2_e.bounds = (-18.5, 0.0)

# get the carbon sources from the model
listPotCarbonSources=[]
for r in m.reactions:
    if 'EX_' in r.id:
        for met in r.metabolites:
            if 'C' in met.formula:
                listPotCarbonSources.append(r.id)
                
m.reactions.EX_glc__D_e.lower_bound = 0.0
m.reactions.get_by_id('EX_o2_e').lower_bound = -18.5
medium = m.medium





CgrowthCapabilities = pd.DataFrame(columns=listPotCarbonSources)

# get values for WT
col = []
for source in listPotCarbonSources:
    m.reactions.get_by_id(source).lower_bound=-10.0
    
    try:
        sol_bio = m.optimize()
        if sol_bio.status != 'infeasible':
            col.append(sol_bio['BIOMASS_Ec_iML1515_core_75p37M'])
        else:
            col.append(0.0)

    except:
        col.append(0.0)
    
    CgrowthCapabilities.insert(0, column = 'WT', value = col)


# run for all knockouts
count = 1
for gene in genes_ALL:
    
    with m: 
        col = []
        m.genes.get_by_id(gene).knock_out() 
        
        for source in listPotCarbonSources:
            m.reactions.get_by_id(source).lower_bound=-10.0


            try:
                sol_bio = m.optimize()
                if sol_bio.status != 'infeasible':
                    col.append(sol_bio['BIOMASS_Ec_iML1515_core_75p37M'])
                else:
                    col.append(0.0)

            except:
                col.append(0.0)
    
    

        # turn transporter off
        if source not in medium.keys():
            m.reactions.get_by_id(source).lower_bound = 0.0
        
        CgrowthCapabilities.insert(count, column = gene_name_dic2[gene], value = col)


